# Assignment 4: Model Monitoring
## Peter Ye

In [ ]:
import pandas as pd
import numpy as np

from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score

from evidently import ColumnMapping
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset, RegressionPreset

## Load Data

In [2]:
CancerData = pd.read_csv('cancer_reg.csv', encoding='latin')

## EDA

### Check data

In [3]:
CancerData.head()

,avgAnnCount,avgDeathsPerYear,TARGET_deathRate,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,binnedInc,MedianAge,...,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate
0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",39.3,...,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",33.0,...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",45.0,...,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488
3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",42.8,...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",48.3,...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657


## 

In [4]:
CancerData.shape

(3047, 34)

In [5]:
CancerData.columns

Index(['avgAnnCount', 'avgDeathsPerYear', 'TARGET_deathRate', 'incidenceRate',
       'medIncome', 'popEst2015', 'povertyPercent', 'studyPerCap', 'binnedInc',
       'MedianAge', 'MedianAgeMale', 'MedianAgeFemale', 'Geography',
       'AvgHouseholdSize', 'PercentMarried', 'PctNoHS18_24', 'PctHS18_24',
       'PctSomeCol18_24', 'PctBachDeg18_24', 'PctHS25_Over',
       'PctBachDeg25_Over', 'PctEmployed16_Over', 'PctUnemployed16_Over',
       'PctPrivateCoverage', 'PctPrivateCoverageAlone', 'PctEmpPrivCoverage',
       'PctPublicCoverage', 'PctPublicCoverageAlone', 'PctWhite', 'PctBlack',
       'PctAsian', 'PctOtherRace', 'PctMarriedHouseholds', 'BirthRate'],
      dtype='object')

In [6]:
CancerData.describe()

,avgAnnCount,avgDeathsPerYear,TARGET_deathRate,incidenceRate,medIncome,popEst2015,povertyPercent,studyPerCap,MedianAge,MedianAgeMale,...,PctPrivateCoverageAlone,PctEmpPrivCoverage,PctPublicCoverage,PctPublicCoverageAlone,PctWhite,PctBlack,PctAsian,PctOtherRace,PctMarriedHouseholds,BirthRate
count,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3.047000e+03,3047.000000,3047.000000,3047.000000,3047.000000,...,2438.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000,3047.000000
mean,606.338544,185.965868,178.664063,448.268586,47063.281917,1.026374e+05,16.878175,155.399415,45.272333,39.570725,...,48.453774,41.196324,36.252642,19.240072,83.645286,9.107978,1.253965,1.983523,51.243872,5.640306
std,1416.356223,504.134286,27.751511,54.560733,12040.090836,3.290592e+05,6.409087,529.628366,45.304480,5.226017,...,10.083006,9.447687,7.841741,6.113041,16.380025,14.534538,2.610276,3.517710,6.572814,1.985816
min,6.000000,3.000000,59.700000,201.300000,22640.000000,8.270000e+02,3.200000,0.000000,22.300000,22.400000,...,15.700000,13.500000,11.200000,2.600000,10.199155,0.000000,0.000000,0.000000,22.992490,0.000000
25%,76.000000,28.000000,161.200000,420.300000,38882.500000,1.168400e+04,12.150000,0.000000,37.700000,36.350000,...,41.000000,34.500000,30.900000,14.850000,77.296180,0.620675,0.254199,0.295172,47.763063,4.521419
50%,171.000000,61.000000,178.100000,453.549422,45207.000000,2.664300e+04,15.900000,0.000000,41.000000,39.600000,...,48.700000,41.100000,36.300000,18.800000,90.059774,2.247576,0.549812,0.826185,51.669941,5.381478
75%,518.000000,149.000000,195.200000,480.850000,52492.000000,6.867100e+04,20.400000,83.650776,44.000000,42.500000,...,55.600000,47.700000,41.550000,23.100000,95.451693,10.509732,1.221037,2.177960,55.395132,6.493677
max,38150.000000,14010.000000,362.800000,1206.900000,125635.000000,1.017029e+07,47.400000,9762.308998,624.000000,64.700000,...,78.900000,70.700000,65.100000,46.600000,100.000000,85.947799,42.619425,41.930251,78.075397,21.326165


In [7]:
# Check null value
CancerData.isna().sum()

avgAnnCount                   0
avgDeathsPerYear              0
TARGET_deathRate              0
incidenceRate                 0
medIncome                     0
popEst2015                    0
povertyPercent                0
studyPerCap                   0
binnedInc                     0
MedianAge                     0
MedianAgeMale                 0
MedianAgeFemale               0
Geography                     0
AvgHouseholdSize              0
PercentMarried                0
PctNoHS18_24                  0
PctHS18_24                    0
PctSomeCol18_24            2285
PctBachDeg18_24               0
PctHS25_Over                  0
PctBachDeg25_Over             0
PctEmployed16_Over          152
PctUnemployed16_Over          0
PctPrivateCoverage            0
PctPrivateCoverageAlone     609
PctEmpPrivCoverage            0
PctPublicCoverage             0
PctPublicCoverageAlone        0
PctWhite                      0
PctBlack                      0
PctAsian                      0
PctOther

### Drop unnecessary columns

In [8]:
CancerData.drop(['PctPrivateCoverageAlone', 'PctSomeCol18_24', 'Geography'], axis=1, inplace=True)

### Check missing value

In [9]:
CancerData.isna().sum()

avgAnnCount                 0
avgDeathsPerYear            0
TARGET_deathRate            0
incidenceRate               0
medIncome                   0
popEst2015                  0
povertyPercent              0
studyPerCap                 0
binnedInc                   0
MedianAge                   0
MedianAgeMale               0
MedianAgeFemale             0
AvgHouseholdSize            0
PercentMarried              0
PctNoHS18_24                0
PctHS18_24                  0
PctBachDeg18_24             0
PctHS25_Over                0
PctBachDeg25_Over           0
PctEmployed16_Over        152
PctUnemployed16_Over        0
PctPrivateCoverage          0
PctEmpPrivCoverage          0
PctPublicCoverage           0
PctPublicCoverageAlone      0
PctWhite                    0
PctBlack                    0
PctAsian                    0
PctOtherRace                0
PctMarriedHouseholds        0
BirthRate                   0
dtype: int64

In [10]:
# Fill it with median value
CancerData['PctEmployed16_Over'] = CancerData['PctEmployed16_Over'].fillna(CancerData['PctEmployed16_Over'].median())

### Mapping the object variable to Numeric: 'binnedInc'

In [11]:
MapResult = {'(61494.5, 125635]':10, '(48021.6, 51046.4]':7, '(42724.4, 45201]':5,
       '(51046.4, 54545.6]':8, '(37413.8, 40362.7]':3, '(40362.7, 42724.4]':4,
       '(54545.6, 61494.5]':9, '(34218.1, 37413.8]':2, '[22640, 34218.1]':1,
       '(45201, 48021.6]':6}

In [12]:
CancerData['binnedInc'] = CancerData['binnedInc'].map(MapResult)

### Select predictors and target variable

In [40]:
Predictors = ['PctBachDeg25_Over', 'incidenceRate',
       'PctPublicCoverageAlone', 'povertyPercent', 'medIncome',
       'PctEmployed16_Over', 'PctHS25_Over', 'PctPublicCoverage',
       'PctPrivateCoverage', 'PctUnemployed16_Over', 'PctMarriedHouseholds',
       'PctBachDeg18_24', 'PctEmpPrivCoverage', 'PercentMarried', 'PctHS18_24',
       'PctBlack', 'binnedInc', 'AvgHouseholdSize']
Target = ['TARGET_deathRate']
X = CancerData[Predictors].values
y = CancerData[Target].values

## Train a model

### Split data 

In [41]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42)

In [42]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(2437, 18)
(2437, 1)
(610, 18)
(610, 1)


### Adaboost

In [43]:
# Ensure y_train and y_test are 1D
y_train = y_train.ravel()
y_test = y_test.ravel()

# Train model
dt = DecisionTreeRegressor(max_depth=10)
adab = AdaBoostRegressor(n_estimators=200, estimator=dt, learning_rate=0.01)
adabRegModel = adab.fit(X_train, y_train)
adabPrediction = adabRegModel.predict(X_test)

print('R2 value', r2_score(y_train, adabRegModel.predict(X_train)))
print('Accuracy', 100 - (np.mean((np.abs(y_test - adabPrediction) / y_test)) * 100))

R2 value 0.8824072553949001
Accuracy 91.69603339929691


## Model Monitoring

### initial report

In [45]:
# Create DataFrames with column names
X_train_df = pd.DataFrame(X_train, columns=Predictors)
X_test_df = pd.DataFrame(X_test, columns=Predictors)
y_train_df = pd.DataFrame(y_train, columns=Target)
y_test_df = pd.DataFrame(y_test, columns=Target)

# Add predictions to DataFrames
train_data = X_train_df.copy()
train_data['TARGET_deathRate'] = y_train_df
train_data['prediction'] = adabRegModel.predict(X_train)

test_data = X_test_df.copy()
test_data['TARGET_deathRate'] = y_test_df
test_data['prediction'] = adabPrediction

# Set up column mapping for Evidently
column_mapping = ColumnMapping()
column_mapping.target = 'TARGET_deathRate'
column_mapping.prediction = 'prediction'
column_mapping.numerical_features = Predictors

In [46]:
### Generate Initial Evidently Report
report = Report(metrics=[DataDriftPreset(), RegressionPreset()])
report.run(reference_data=train_data, current_data=test_data, column_mapping=column_mapping)
report.save_html('report_initial.html')
print("Initial report saved as 'report_initial.html'.")

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:1211: UndefinedMetricWarning:

R^2 score is not well-defined with less than two samples.

/home/yz

Initial report saved as 'report_initial.html'.


### Instance A: Decrease 'medIncome' by $40,000

In [48]:
# Create a copy of the test dataset
X_test_mod1 = X_test_df.copy()
X_test_mod1['medIncome'] = X_test_mod1['medIncome'] - 40000
X_test_mod1['medIncome'] = X_test_mod1['medIncome'].clip(lower=0)  # Ensure no negative incomes

# Run the model on modified data
adabPrediction_mod1 = adabRegModel.predict(X_test_mod1)

# Evaluate the accuracy
accuracy_mod1 = 100 - (np.mean((np.abs(y_test - adabPrediction_mod1) / y_test)) * 100)
print('Accuracy after modification A:', accuracy_mod1)

# Prepare data for Evidently
test_data_mod1 = X_test_mod1.copy()
test_data_mod1['TARGET_deathRate'] = y_test_df
test_data_mod1['prediction'] = adabPrediction_mod1

Accuracy after modification A: 91.00766014399844


/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but AdaBoostRegressor was fitted without feature names



In [49]:
# Generate Evidently report
report_mod1 = Report(metrics=[DataDriftPreset(), RegressionPreset()])
report_mod1.run(reference_data=test_data, current_data=test_data_mod1, column_mapping=column_mapping)
report_mod1.save_html('report_mod1.html')
print("Report after modification A saved as 'report_mod1.html'.")

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calcula

Report after modification A saved as 'report_mod1.html'.


### Instance A & B: Decrease 'medIncome' and Increase 'povertyPercent' by 20 Points

In [50]:
# Create a copy of the modified dataset from Instance A
X_test_mod2 = X_test_mod1.copy()
X_test_mod2['povertyPercent'] = X_test_mod2['povertyPercent'] + 20
X_test_mod2['povertyPercent'] = X_test_mod2['povertyPercent'].clip(upper=100)  # Ensure max 100%

# Run the model on modified data
adabPrediction_mod2 = adabRegModel.predict(X_test_mod2)

# Evaluate the accuracy
accuracy_mod2 = 100 - (np.mean((np.abs(y_test - adabPrediction_mod2) / y_test)) * 100)
print('Accuracy after modification A & B:', accuracy_mod2)

# Prepare data for Evidently
test_data_mod2 = X_test_mod2.copy()
test_data_mod2['TARGET_deathRate'] = y_test_df
test_data_mod2['prediction'] = adabPrediction_mod2

Accuracy after modification A & B: 90.30134269534997


/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but AdaBoostRegressor was fitted without feature names



In [51]:
# Generate Evidently report
report_mod2 = Report(metrics=[DataDriftPreset(), RegressionPreset()])
report_mod2.run(reference_data=test_data, current_data=test_data_mod2, column_mapping=column_mapping)
report_mod2.save_html('report_mod2.html')
print("Report after modifications A & B saved as 'report_mod2.html'.")

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calcula

Report after modifications A & B saved as 'report_mod2.html'.


### Instance A, B & C: Also Increase 'avgHouseholdSize' by 2

In [52]:
# Create a copy of the modified dataset from Instance A & B
X_test_mod3 = X_test_mod2.copy()
X_test_mod3['AvgHouseholdSize'] = X_test_mod3['AvgHouseholdSize'] + 2

# Run the model on modified data
adabPrediction_mod3 = adabRegModel.predict(X_test_mod3)

# Evaluate the accuracy
accuracy_mod3 = 100 - (np.mean((np.abs(y_test - adabPrediction_mod3) / y_test)) * 100)
print('Accuracy after modifications A, B & C:', accuracy_mod3)

# Prepare data for Evidently
test_data_mod3 = X_test_mod3.copy()
test_data_mod3['TARGET_deathRate'] = y_test_df
test_data_mod3['prediction'] = adabPrediction_mod3

Accuracy after modifications A, B & C: 90.49734203560071


/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/base.py:486: UserWarning:

X has feature names, but AdaBoostRegressor was fitted without feature names



In [53]:
# Generate Evidently report
report_mod3 = Report(metrics=[DataDriftPreset(), RegressionPreset()])
report_mod3.run(reference_data=test_data, current_data=test_data_mod3, column_mapping=column_mapping)
report_mod3.save_html('report_mod3.html')
print("Report after modifications A, B & C saved as 'report_mod3.html'.")

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.

/home/yzysnake/miniconda3/envs/Mlop/lib/python3.9/site-packages/sklearn/metrics/_regression.py:492: FutureWarning:

'squared' is deprecated in version 1.4 and will be removed in 1.6. To calcula

Report after modifications A, B & C saved as 'report_mod3.html'.
